# 続・detectron2 for まちカドまぞく ～カスタム訓練編～

<img src="https://user-images.githubusercontent.com/33882378/79110283-0f737980-7db5-11ea-89b0-3b6b19ae1716.jpg">

detectron2 の訓練をカスタマイズする方法

ここでは、「カスタムトレーナー」を作りますが、今までの工程では一番楽でした。。

---

In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import random

---
##  VoTT Export からの読み込み

In [2]:
# VoTT のエクスポートファイルや、画像が格納されているディレクトリ
BASE_DIRECTORY = './vott-json-export/'
# VoTT のエクスポートファイル名
EXPORT_FILENAME = 'Machikado-export.json'
# 訓練データに使用する割合
TRAIN_RATIO = 0.8
# 乱数シード
RANDOM_STATE = 0

In [3]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from machikado_util.Machikado_vott import get_cat_names, get_machikado_dicts

# vott エクスポートファイルの読み込み
CAT_NAME2ID, CAT_ID2NAME = get_cat_names(os.path.join(BASE_DIRECTORY, EXPORT_FILENAME))
dataset_dicts = get_machikado_dicts(os.path.join(BASE_DIRECTORY, EXPORT_FILENAME), BASE_DIRECTORY, CAT_NAME2ID)

# 訓練用、テスト用に分ける
random.seed(RANDOM_STATE)
random.shuffle(dataset_dicts)

split_idx = int(len(dataset_dicts) * TRAIN_RATIO) + 1

# 登録
DatasetCatalog.clear()
DatasetCatalog.register('train', lambda : dataset_dicts[:split_idx])
DatasetCatalog.register('test', lambda : dataset_dicts[split_idx:])

MetadataCatalog.get('train').set(thing_classes=list(CAT_NAME2ID.keys()))
MetadataCatalog.get('test').set(thing_classes=list(CAT_NAME2ID.keys()))

警告: name: 59.jpg - 画像サイズが不一致であるためスキップ image_size:(268, 201), ./vott-json-export/Machikado-export.json: (600, 600)


Metadata(name='test', thing_classes=['Shamiko', 'Gosenzo', 'Lilith', 'Momo', 'Mikan', 'Mob'])

In [4]:
from detectron2.data import DatasetCatalog

# 訓練用、テスト用に分ける
random.seed(RANDOM_STATE)
random.shuffle(dataset_dicts)
split_idx = int(len(dataset_dicts) * TRAIN_RATIO) + 1

# 登録
DatasetCatalog.clear()
DatasetCatalog.register('train', lambda : dataset_dicts[:split_idx])
DatasetCatalog.register('test', lambda : dataset_dicts[split_idx:])

---
## 学習

* 前回より訓練回数を盛っています。

In [5]:
from detectron2.config import get_cfg
from machikado_util.custom_config import append_custom_cfg

cfg = get_cfg()
append_custom_cfg(cfg)

cfg.OUTPUT_DIR = './output'
cfg.CUDA = 'cuda:0'

# cfg.merge_from_file("../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.WEIGHTS = './coco_models/model_final_f10217.pkl'
# cfg.SOLVER.IMS_PER_BATCH = 2

# 重いけど、これ精度良いです。
cfg.merge_from_file('../configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml')
cfg.MODEL.WEIGHTS = './coco_models/model_final_2d9806.pkl'
cfg.SOLVER.IMS_PER_BATCH = 1 # GTX2070 ではこれが限界

cfg.DATASETS.TRAIN = ('train',)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(CAT_ID2NAME) 

cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.SIZE = [0.8, 0.8]

#### カスタムトレーナー

* DefaultTrainer は detectron2/engine/defaults.py にあります。
* DefaultTrainer を全部書き換える必要は無く、今回の場合であればデーターマッパーを渡す部分を変更するだけで良いです。（オーバーライドすれば良いです。）
* あとは、同じです。

In [6]:
from machikado_util.MachikadoDatasetMapper import MachikadoDatasetMapper
from detectron2.engine import DefaultTrainer

class MachikadoTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
    
    # ここをオーバーライドして、作った自分のデータマッパーを返すように変更する。
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, MachikadoDatasetMapper(cfg))

In [7]:
from detectron2.data import build_detection_test_loader, build_detection_train_loader

# 出力先のディレクトリを作る
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = MachikadoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[04/17 03:45:05 d2.data.detection_utils]: TransformGens used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/17 03:45:05 d2.data.build]: Removed 0 images with no usable annotations. 92 images left.
[04/17 03:45:05 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|  Shamiko   | 74           |  Gosenzo   | 28           |   Lilith   | 11           |
|    Momo    | 42           |   Mikan    | 15           |    Mob     | 19           |
|            |              |            |              |            |              |
|   total    | 189          |            |              |            |              |
[04/17 03:45:05 d2.data.common]: Serializing 92 elements to byte tensors and concatenating them all ...
[04/17 0

'roi_heads.box_predictor.cls_score.weight' has shape (81, 1024) in the checkpoint but (7, 1024) in the model! Skipped.
'roi_heads.box_predictor.cls_score.bias' has shape (81,) in the checkpoint but (7,) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.weight' has shape (320, 1024) in the checkpoint but (24, 1024) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.bias' has shape (320,) in the checkpoint but (24,) in the model! Skipped.
'roi_heads.mask_head.predictor.weight' has shape (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) in the model! Skipped.
'roi_heads.mask_head.predictor.bias' has shape (80,) in the checkpoint but (6,) in the model! Skipped.


[04/17 03:45:06 d2.engine.train_loop]: Starting training from iteration 0
[04/17 03:45:15 d2.utils.events]:  eta: 0:18:57  iter: 19  total_loss: 3.368  loss_cls: 2.100  loss_box_reg: 0.510  loss_mask: 0.694  loss_rpn_cls: 0.002  loss_rpn_loc: 0.013  time: 0.4581  data_time: 0.0087  lr: 0.000005  max_mem: 3785M
[04/17 03:45:25 d2.utils.events]:  eta: 0:19:35  iter: 39  total_loss: 3.591  loss_cls: 1.960  loss_box_reg: 0.929  loss_mask: 0.692  loss_rpn_cls: 0.006  loss_rpn_loc: 0.016  time: 0.4711  data_time: 0.0016  lr: 0.000010  max_mem: 3785M
[04/17 03:45:35 d2.utils.events]:  eta: 0:19:40  iter: 59  total_loss: 3.213  loss_cls: 1.657  loss_box_reg: 0.810  loss_mask: 0.690  loss_rpn_cls: 0.005  loss_rpn_loc: 0.018  time: 0.4734  data_time: 0.0015  lr: 0.000015  max_mem: 3785M
[04/17 03:45:44 d2.utils.events]:  eta: 0:19:16  iter: 79  total_loss: 2.513  loss_cls: 1.319  loss_box_reg: 0.594  loss_mask: 0.684  loss_rpn_cls: 0.004  loss_rpn_loc: 0.015  time: 0.4686  data_time: 0.0017  lr: